In [1]:
!nvidia-smi

Thu Apr 11 14:46:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         On | 00000000:23:00.0 Off |                  N/A |
| 30%   33C    P8               23W / 350W|      1MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from src.segmentation.framework_handlers import yolo_handler
yolo_handler.validate_version_and_gpu()

Ultralytics YOLOv8.1.46 🚀 Python-3.10.0 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Setup complete ✅ (64 CPUs, 503.6 GB RAM, 58.8/788.3 GB disk)


### Prepare Dataset
In this section, the dataset along with YOLO annotations is processed to conform to the input structure required by YOLO. The YOLO format necessitates organizing the dataset into two main directories, `images` and `labels`. Each of these directories is further divided into two subdirectories, `train` and `val`, for training and validation datasets respectively. This structure facilitates YOLO's training and validation processes.

In [3]:
dataset_name = "etaylor/cannabis_patches_all_11_4_24"
dataset_version = "v0.1"

In [4]:
from src.annotation_handling import annotation_handler

# if the dataset was not exported from segments api, use this function to export it and then convert it to yolo format.
annotation_handler.convert_segments_to_coco_format(dataset_name=dataset_name,
                                                    release_version=dataset_version)

Initializing dataset...
Preloading all samples. This may take a while...


100%|██████████| 228/228 [00:06<00:00, 33.41it/s]


Initialized dataset with 228 images.
Exporting dataset. This may take a while...


100%|██████████| 228/228 [00:04<00:00, 46.01it/s]

Exported to ./export_coco-instance_etaylor_cannabis_patches_all_11_4_24_v0.1.json. Images in segments/etaylor_cannabis_patches_all_11_4_24/v0.1


(<segments.dataset.SegmentsDataset at 0x7f5e42f99510>,
 'segments/etaylor_cannabis_patches_all_11_4_24/annotations/export_coco-instance_etaylor_cannabis_patches_all_11_4_24_v0.1.json',
 'segments/etaylor_cannabis_patches_all_11_4_24/v0.1')

In [7]:
# prepare data to fine tune yolo v8
annotations_folder_name = dataset_name.replace("/", "_")

saving_yaml_path = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/yaml_files"
yaml_file_path = annotation_handler.convert_coco_to_yolo_format(annotations_folder_name, dataset_version, saving_yaml_path, train_percentage=0.8)

Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_all_11_4_24/annotations/export_coco-instance_etaylor_cannabis_patches_all_11_4_24_v0.1.json:   0%|          | 0/226 [00:00<?, ?it/s]

Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_all_11_4_24/annotations/export_coco-instance_etaylor_cannabis_patches_all_11_4_24_v0.1.json: 100%|██████████| 226/226 [00:00<00:00, 618.07it/s]

COCO data converted successfully.
Results saved to /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_all_11_4_24/annotations/yolo


The folder /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_all_11_4_24/annotations/yolo has been deleted.


### Ultralytics Settings

In here we can modify the settings of the yolo runs. 

In [4]:
from ultralytics import settings

runs_dir = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/runs"
# Update a setting
settings.update({'runs_dir': runs_dir})

# View all settings
print(settings)

{'settings_version': '0.0.4', 'datasets_dir': '/sise/home/etaylor/datasets', 'weights_dir': '/home/etaylor/weights', 'runs_dir': '/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/runs', 'uuid': '780eeb8bcf4edc15ebd27762705ae2df2741deb0dd4dddfa1ad6c77841a3348c', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}


### Fine tune YOLOv8 Model on my dataset

In [5]:
# in case dataset already converted from coco to yolo format use manual path
yaml_file_path = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/yaml_files/etaylor_cannabis_patches_test_v0.1_data.yaml"


In [7]:
from ultralytics import RTDETR

model_checkpoint_path = 'rtdetr-x.pt'
model = RTDETR(model_checkpoint_path)

100%|██████████| 129M/129M [00:03<00:00, 42.7MB/s] 


In [8]:
# regular train of the model
results = model.train(data=yaml_file_path, epochs=100, imgsz=512)

Ultralytics YOLOv8.1.46 🚀 Python-3.10.0 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=rtdetr-x.pt, data=/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/yaml_files/etaylor_cannabis_patches_test_v0.1_data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train40, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=F

train: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo_split/labels/train.cache... 73 images, 70 backgrounds, 0 corrupt: 100%|██████████| 143/143 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo_split/labels/val.cache... 16 images, 20 backgrounds, 0 corrupt: 100%|██████████| 36/36 [00:00<?, ?it/s]


Plotting labels to /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/runs/detect/train40/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 193 weight(decay=0.0), 256 weight(decay=0.0005), 276 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/runs/detect/train40
Starting training for 100 epochs...

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


...

  0%|          | 0/9 [00:00<?, ?it/s]/home/etaylor/.conda/envs/yolov9/lib/python3.10/site-packages/torch/autograd/__init__.py:266: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:79.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
    100/100      13.5G     0.2327     0.3711    0.05492        125        512: 100%|██████████| 9/9 [00:04<00:00,  2.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.01it/s]

                   all         36        167      0.621      0.598      0.577      0.401



100 epochs completed in 0.184 hours.
Optimizer stripped from /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/runs/detect/train40/weights/last.pt, 135.4MB
Optimizer stripped from /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/runs/detect/train40/weights/best.pt, 135.4MB

Validating /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/runs/detect/train40/weights/best.pt...
Ultralytics YOLOv8.1.46 🚀 Python-3.10.0 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
rt-detr-x summary: 642 layers, 65473601 parameters, 0 gradients, 222.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.91it/s]


                   all         36        167      0.601      0.605      0.581      0.419
                 clear         36         47      0.567      0.511      0.554      0.376
                cloudy         36        106      0.697       0.66      0.662      0.488
                 amber         36         14      0.539      0.643      0.529      0.392
Speed: 0.2ms preprocess, 6.2ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO/runs/detect/train40


In [8]:
valid_results = model.val()
print(valid_results)

Ultralytics YOLOv8.1.46 🚀 Python-3.10.0 torch-2.2.2+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)


YOLOv9c summary (fused): 384 layers, 25321561 parameters, 0 gradients, 102.3 GFLOPs


val: Scanning /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo_split/labels/val.cache... 16 images, 20 backgrounds, 0 corrupt: 100%|██████████| 36/36 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.10it/s]


                   all         36        167      0.538      0.688      0.639      0.431
                 clear         36         47      0.478       0.66      0.616      0.411
                cloudy         36        106      0.687      0.705      0.723      0.515
                 amber         36         14      0.449      0.698      0.579      0.367
Speed: 0.1ms preprocess, 23.4ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/runs/detect/train382
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f0a41f88700>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.0070

In [ ]:
from IPython.display import Image as show_image
show_image(filename="/sise/home/etaylor/runs/segment/train22/val_batch0_pred.jpg")

### Perform Infrence with the model

In [8]:
import cv2

image = cv2.imread("/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_train/v0.1/IMG_2198_p12.png")

# Perform inference
results = model.predict(image)


0: 640x640 (no detections), 19.5ms
Speed: 12.6ms preprocess, 19.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
